<a href="https://colab.research.google.com/github/ragalaa/Machine-Learning-Basics/blob/master/Recommendation_Engine_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Recommendation Engine from scrath using the Movielens Dataset

In [0]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [15]:
## Users data set:

# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.user
##Ratings datset
# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.data
### Items datset
# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.item

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  230k  100  230k    0     0   894k      0 --:--:-- --:--:-- --:--:--  891k


In [6]:
# Reading file and passing columns names

col_users = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('./u.user',sep='|', names=col_users)

col_rat = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('./u.data',sep='\t', names=col_rat)

col_items = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('./u.item',sep='|', names=col_items, encoding='latin-1')

NameError: ignored

EDA ( Exploratory data analysis) with shape and **duplicates**

In [32]:
print('users'+ str(users.shape))
users.head()


users(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [33]:
# Checking duplicates
print((users.shape[0] - len([x for x in (users.user_id.value_counts() > 1).values if x == False]))/2)

0.0


In [21]:
print('Ratings'+ str(ratings.shape))
ratings.head()

Ratings(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [22]:
print('Items'+ str(items.shape))
items.head()

Items(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [5]:
# Checking non repeated-movies
print((items.shape[0] - len([x for x in (items.movie_id.value_counts() > 1).values if x == False]))/2)

NameError: ignored

In [4]:
# First we need to crete a user-item matrix to be used 
# to calculate the similarity between users and items.

n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
print( n_users, n_items)

# We'll create a matrix full of zeros
# And then we'll go by everyline in ratings, filling the user row
# and the movie column, with the rating.

data_matrix = np.zeros((n_users, n_items))

for line in ratings.itertuples():
    print(line)
    break
#     data_matrix[line[1]-1, line[2]-1] = line[3]

NameError: ignored